In [38]:
import pandas as pd
from IPython.display import display, HTML
import ipywidgets as widgets
from google.colab import files

# Carrega a planilha (CSV ou XLSX)
def carregar_planilha(nome_arquivo):
    if nome_arquivo.endswith('.csv'):
        return pd.read_csv(nome_arquivo)
    elif nome_arquivo.endswith('.xlsx'):
        return pd.read_excel(nome_arquivo)
    else:
        print("Formato não suportado. Use .csv ou .xlsx.")
        return None

# Função principal
def realizar_sorteio():
    uploaded = files.upload()
    nome_arquivo = list(uploaded.keys())[0]
    df = carregar_planilha(nome_arquivo)
    if df is None:
        return

    participantes = df.copy()
    sorteados = []

    output = widgets.Output()
    continuar_dropdown = widgets.Dropdown(
        options=[('Sim', 's'), ('Não', 'n')],
        description='Novo sorteio?',
        value=None,
        layout=widgets.Layout(width='300px')
    )
    continuar_dropdown.layout.display = 'none'

    botao_sorteio = widgets.Button(
        description="🎁 Sortear Brinde",
        layout=widgets.Layout(width='200px'),
        button_style='success'
    )

    # Exibe tabela inicial dos participantes
    def mostrar_participantes():
        display(HTML("""
        <div style="margin-top:20px">
            <h3 style="color:#333;">🎯 Participantes na lista:</h3>
        </div>"""))
        display(HTML(df[['Nome', 'Telefone', 'Email']].to_html(index=False,
               classes='table table-striped table-bordered', border=0)))

    # Exibe sorteados até agora
    def mostrar_sorteados():
        if sorteados:
            nomes = "".join(f"<li>🎉 {nome}</li>" for nome in sorteados)
            display(HTML(f"""
            <div style="margin-top:20px;">
                <h4 style="color:#4CAF50;">✅ Sorteadas até agora:</h4>
                <ul style="font-size:16px; color:#333;">{nomes}</ul>
            </div>"""))

    # Sorteia um nome
    def sortear():
        nonlocal participantes
        if len(participantes) == 0:
            with output:
                output.clear_output()
                display(HTML("<h4 style='color:#d9534f;'>Todas os participantes já foram sorteadas! 🏁</h4>"))
                mostrar_sorteados()
            botao_sorteio.disabled = True
            continuar_dropdown.layout.display = 'none'
            return

        sorteado = participantes.sample(1)
        nome = sorteado.iloc[0]['Nome']
        sorteados.append(nome)
        participantes = participantes.drop(sorteado.index).reset_index(drop=True)

        with output:
            output.clear_output()
            display(HTML(f"""
            <div style="padding:10px; border: 2px solid #4CAF50; border-radius:10px; background-color:#e8f5e9;">
                <h3 style="color:#2e7d32;">🎉 Sorteado: <b>{nome}</b></h3>
            </div>"""))
            mostrar_sorteados()
            continuar_dropdown.value = None
            continuar_dropdown.layout.display = 'block'
            display(continuar_dropdown)

    # Evento botão
    def on_botao_click(b):
        botao_sorteio.disabled = True
        sortear()

    # Evento dropdown
    def on_dropdown_change(change):
        if change['new'] == 's':
            botao_sorteio.disabled = False
            sortear()
        elif change['new'] == 'n':
            with output:
                output.clear_output()
                display(HTML("""
                <div style="padding:15px; border: 2px solid #0275d8; border-radius: 8px; background-color: #d9edf7; color: #31708f;">
                    <h3>🏁 Sorteio finalizado. Obrigado por participar!</h3>
                    <h4>🎉 Nomes sorteados:</h4>
                    <ul style="font-size:16px;">
                        {}
                    </ul>
                </div>""".format("".join(f"<li>{nome}</li>" for nome in sorteados))))
            botao_sorteio.disabled = True
            continuar_dropdown.layout.display = 'none'

    botao_sorteio.on_click(on_botao_click)
    continuar_dropdown.observe(on_dropdown_change, names='value')

    # Exibição inicial
    display(HTML("""
    <div style="background-color:#f1f8e9; padding:15px; border-radius:10px; margin-bottom:10px;">
        <h2 style="color:#33691e;">🎊 Bem-vindas ao Sorteio das Pyladies!</h2>
        <p style="font-size:16px;">Clique no botão abaixo para começar a sortear os participantes.</p>
    </div>"""))

    mostrar_participantes()
    display(botao_sorteio)
    display(output)

# Executar
realizar_sorteio()



Saving teste_plan.xlsx to teste_plan (34).xlsx


Nome,Telefone,Email
João da Silva,(48) 99999-1234,joao@email.com
Maria Oliveira,(48) 98888-5678,maria@email.com
Pedro Souza,(48) 97777-3456,pedro@email.com
Ana Costa,(48) 96666-7890,ana@email.com
Carlos Pereira,(48) 95555-4321,carlos@email.com
Luciana Ferreira,(48) 94444-6789,luciana@email.com
Roberto Martins,(48) 93333-1234,roberto@email.com
Fernanda Lima,(48) 92222-4567,fernanda@email.com
Paulo Henrique,(48) 91111-6789,paulo@email.com
Carolina Silva,(48) 90000-1234,carolina@email.com


Button(button_style='success', description='🎁 Sortear Brinde', layout=Layout(width='200px'), style=ButtonStyle…

Output()